In [ ]:
import sys
repo_path = '/home/micael.verissimo/paper_lzt/exp-connection-hep/'
sys.path.insert(0, repo_path)

In [2]:
import os
import tqdm
import pickle
import collections
import itertools

import matplotlib as mpl
mpl.rcParams['text.usetex'] = True

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
tf.config.run_functions_eagerly(True)
tf.get_logger().setLevel('INFO')

from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from loguru import logger

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

from src.crossval import crossval_table

2025-07-11 01:06:45.622657: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-11 01:06:45.622704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-11 01:06:45.623773: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-11 01:06:45.630406: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-11 01:06:48.147898: W tensorflow/compiler/tf2

Welcome to JupyROOT 6.30/02


2025-07-11 01:06:57.718036: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-11 01:06:57.744222: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-11 01:06:57.744512: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
#
# Create the reference map
#
def create_op_dict(op, decoration='reference'):
    
    d = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              
              # op
              'pd_ref'    : decoration+"/"+op+"/pd_ref#0",
              #'fa_ref'    : decoration+"/"+op+"/fa_ref#0",
              #'sp_ref'    : decoration+"/"+op+"/sp_ref",
              'pd_val'    : decoration+"/"+op+"/pd_val#0",
              'fa_val'    : decoration+"/"+op+"/fa_val#0",
              'thr_val'   : decoration+"/"+op+"/threshold_val",
              'sp_val'    : decoration+"/"+op+"/sp_val",
              'pd_op'     : decoration+"/"+op+"/pd_op#0",
              'fa_op'     : decoration+"/"+op+"/fa_op#0",
              'sp_op'     : decoration+"/"+op+"/sp_op",
              'thr_op'    : decoration+"/"+op+"/threshold_op",

              # Counts
              'pd_ref_passed'    : decoration+"/"+op+"/pd_ref#1",
              #'fa_ref_passed'    : decoration+"/"+op+"/fa_ref#1",
              'pd_ref_total'     : decoration+"/"+op+"/pd_ref#2",
              #'fa_ref_total'     : decoration+"/"+op+"/fa_ref#2",
              'pd_val_passed'    : decoration+"/"+op+"/pd_val#1",
              'fa_val_passed'    : decoration+"/"+op+"/fa_val#1",
              'pd_val_total'     : decoration+"/"+op+"/pd_val#2",
              'fa_val_total'     : decoration+"/"+op+"/fa_val#2",
              'pd_op_passed'     : decoration+"/"+op+"/pd_op#1",
              'fa_op_passed'     : decoration+"/"+op+"/fa_op#1",
              'pd_op_total'      : decoration+"/"+op+"/pd_op#2",
              'fa_op_total'      : decoration+"/"+op+"/fa_op#2",
    })
    return d


op_names = ['tight', 'medium', 'loose']

tuned_info = collections.OrderedDict({})
for op in op_names:
    tuned_info[op] = create_op_dict(op, "reference")

In [4]:
#
# Et/eta bins edges
# 
etbins = [15, 30, 50, 100000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]
tuning_path = os.path.join(repo_path, 'data/models')

In [5]:
cv_ss    = crossval_table( tuned_info, etbins = etbins, etabins = etabins )
cv_ss.fill( os.path.join(tuning_path, 'ss_models/*.pic'), 'MLP-5-SS')

cv_v0    = crossval_table( tuned_info, etbins = etbins, etabins = etabins )
cv_v0.fill( os.path.join(tuning_path, 'rings_v0_models/*.pic'), 'MLP-5-rings_v0')

cv_v1    = crossval_table( tuned_info, etbins = etbins, etabins = etabins )
cv_v1.fill( os.path.join(tuning_path, 'rings_v1_models/*.pic'), 'MLP-5-rings_v1')

 [Fri, 11 Jul 2025 01:06:58] INFO Reading file for MLP-5-SS tag from /home/micael.verissimo/paper_lzt/exp-connection-hep/data/models/ss_models/*.pic
 [Fri, 11 Jul 2025 01:06:58] INFO Reading file for MLP-5-SS tag from /home/micael.verissimo/paper_lzt/exp-connection-hep/data/models/ss_models/*.pic
 [Fri, 11 Jul 2025 01:06:58] INFO There are 75 files for this task...
 [Fri, 11 Jul 2025 01:06:58] INFO There are 75 files for this task...
 [Fri, 11 Jul 2025 01:06:58] INFO Filling the table... 
 [Fri, 11 Jul 2025 01:06:58] INFO Filling the table... 
🔥 Reading MLP-5-SS...: 100%|██████████| 75/75 [00:00<00:00, 1162.34it/s]
 [Fri, 11 Jul 2025 01:06:58] INFO End of fill step, a pandas DataFrame was created...
 [Fri, 11 Jul 2025 01:06:58] INFO End of fill step, a pandas DataFrame was created...
 [Fri, 11 Jul 2025 01:06:58] INFO Reading file for MLP-5-rings_v0 tag from /home/micael.verissimo/paper_lzt/exp-connection-hep/data/models/rings_v0_models/*.pic
 [Fri, 11 Jul 2025 01:06:58] INFO Reading fi

In [6]:
# obtain the best inits and best sorts and then 
best_inits_ss = cv_ss.filter_inits("max_sp_val")
best_sorts_ss = cv_ss.filter_sorts( best_inits_ss , 'max_sp_op')

best_inits_v0 = cv_v0.filter_inits("max_sp_val")
best_sorts_v0 = cv_v0.filter_sorts( best_inits_v0 , 'max_sp_op')

best_inits_v1 = cv_v1.filter_inits("max_sp_val")
best_sorts_v1 = cv_v1.filter_sorts( best_inits_v1 , 'max_sp_op')

best_inits = pd.concat([
                        best_inits_v0,
                        best_inits_v1,
                        best_inits_ss
                       ])

best_sorts = pd.concat([
                        best_sorts_v0,
                        best_sorts_v1,
                        best_sorts_ss
                       ])

In [7]:
print(len(best_inits))#(15bins x 5folds x 3op x 3inputs = 675)
print(len(best_sorts))#(15bins x 1folds x 3op x 3inputs = 135)

675
135


In [8]:
best_inits.to_csv(os.path.join(repo_path, 'data/processed/best_inits.csv'), index=False)
best_sorts.to_csv(os.path.join(repo_path, 'data/processed/best_sorts.csv'), index=False)

In [8]:
# verificar os tunings separando-os em regiões de low e high energy
cv    = crossval_table( tuned_info, etbins = etbins, etabins = etabins )
for iop in ['tight', 'medium', 'loose']:
    cv.dump_beamer_table( best_inits = best_inits.loc[(best_inits.op_name == iop) & (best_inits.et_bin <= 2)] ,
                          output = f'./cv_{iop}',
                          tags  = ['MLP-5-SS','MLP-5-rings_v0', 'MLP-5-rings_v1'],
                          title = iop+' Tunings LZT', toPDF=False
                        )

 [Thu, 10 Jul 2025 01:12:42] INFO Started creating beamer file ./cv_tight..tex latex code...
 [Thu, 10 Jul 2025 01:12:42] INFO Started creating beamer file ./cv_tight..tex latex code...
 [Thu, 10 Jul 2025 01:12:42] INFO Started creating beamer file ./cv_medium..tex latex code...
 [Thu, 10 Jul 2025 01:12:42] INFO Started creating beamer file ./cv_medium..tex latex code...
 [Thu, 10 Jul 2025 01:12:43] INFO Started creating beamer file ./cv_loose..tex latex code...
 [Thu, 10 Jul 2025 01:12:43] INFO Started creating beamer file ./cv_loose..tex latex code...
